In [49]:
from datetime import datetime

from sqlalchemy import (MetaData, Table, Column, Integer, Numeric, String,
                        DateTime, ForeignKey, Boolean, create_engine)
metadata = MetaData()

cookies = Table('cookies', metadata,
    Column('cookie_id', Integer(), primary_key=True),
    Column('cookie_name', String(50), index=True),
    Column('cookie_recipe_url', String(255)),
    Column('cookie_sku', String(55)),
    Column('quantity', Integer()),
    Column('unit_cost', Numeric(12, 2))
)

users = Table('users', metadata,
    Column('user_id', Integer(), primary_key=True),
    Column('username', String(15), nullable=False, unique=True),
    Column('email_address', String(255), nullable=False),
    Column('phone', String(20), nullable=False),
    Column('password', String(25), nullable=False),
    Column('created_on', DateTime(), default=datetime.now),
    Column('updated_on', DateTime(), default=datetime.now, onupdate=datetime.now)
)

orders = Table('orders', metadata,
    Column('order_id', Integer(), primary_key=True),
    Column('user_id', ForeignKey('users.user_id')),
    # Column('shipped', Boolean(), default=False)
)

line_items = Table('line_items', metadata,
    Column('line_items_id', Integer(), primary_key=True),
    Column('order_id', ForeignKey('orders.order_id')),
    Column('cookie_id', ForeignKey('cookies.cookie_id')),
    Column('quantity', Integer()),
    Column('extended_cost', Numeric(12, 2))
)

engine = create_engine('sqlite:///:memory:')
metadata.create_all(engine)

connection = engine.connect()

In [50]:
# from get_engine import engine, cookies

# # 导入engine
# connection = engine.connect()
# print(connection)


In [51]:

# Example 2-1 Single insert as a method
ins = cookies.insert().values(
    cookie_name='chocolate chip',
    cookie_recipe_url='http://some.aweso.me/cookie/recipe.html',
    cookie_sku='CC01',
    quantity="12",
    unit_cost="0.05"
)
# 查看语句
print(str(ins))
# INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost)
# VALUES (:cookie_name, :cookie_recipe_url, :cookie_sku, :quantity, :unit_cost)

# 查看参数
print(ins.compile().params)
# {'cookie_recipe_url': 'http://some.aweso.me/cookie/recipe.html', 'unit_cost': '0.05',
# 'cookie_sku': 'CC01', 'quantity': '12', 'cookie_name': 'chocolate chip'}

result = connection.execute(ins)

print(result.inserted_primary_key)
# [1]

from sqlalchemy import insert
ins = insert(cookies).values(
    cookie_name="chocolate chip",
    cookie_recipe_url="http://some.aweso.me/cookie/recipe.html",
    cookie_sku="CC01",
    quantity="12",
    unit_cost="0.50"
)
print(str(ins))

ins = cookies.insert()
result = connection.execute(ins, cookie_name='dark chocolate chip',
                            cookie_recipe_url='http://some.aweso.me/cookie/recipe_dark.html',
                            cookie_sku='CC02',
                            quantity='1',
                            unit_cost='0.75'
                            )
print(result.inserted_primary_key)

inventory_list = [
    {
        'cookie_name': 'penaut butter',
        'cookie_recipe_url': 'http://some.aweso.me/cookie/peanut.html',
        'cookie_sku': 'PB01',
        'quantity': '24',
        'unit_cost': '0.25'
    },
    {
        'cookie_name':'oatmeal raisin',
        'cookie_recipe_url': 'http://some.aweso.me/cookie/peanut.html',
        'cookie_sku': '100',
        'quantity': '100',
        'unit_cost': '1.00'

    }
]

result = connection.execute(ins, inventory_list)


# Query data
print('Query_data')

from sqlalchemy import select

s = select([cookies])

print(str(s))

rp = connection.execute(s)
results = rp.fetchall()
print(results)
first_row = results[0]
print(first_row[0])
print(first_row.cookie_name)
print(cookies.c)
print(first_row[cookies.c.cookie_name])

rp = connection.execute(s)
for record in rp:
    print(record.cookie_name)

s = select([cookies.c.cookie_name, cookies.c.quantity])
rp = connection.execute(s)
print(rp.keys())
result = rp.first()
print(result)

# Order
s = select([cookies.c.cookie_name, cookies.c.quantity])
s = s.order_by(cookies.c.quantity)
rp = connection.execute(s)
for cookie in rp:
    print('{} -  {}'.format(cookie.quantity, cookie.cookie_name))

# desc
from sqlalchemy import desc, asc
s = select([cookies.c.cookie_name, cookies.c.quantity])
s = s.order_by(desc(cookies.c.quantity))
rp = connection.execute(s)
for cookie in rp:
    print('{} -  {}'.format(cookie.quantity, cookie.cookie_name))

s = select([cookies.c.cookie_name, cookies.c.quantity])
s = s.order_by(cookies.c.quantity)
s = s.limit(2)
rp = connection.execute(s)
print([result.cookie_name for result in rp])


from sqlalchemy.sql import func
s = select([func.sum(cookies.c.quantity)])
rp = connection.execute(s)
print(rp.scalar())

s = select([func.count(cookies.c.cookie_name)])
rp = connection.execute(s)
record = rp.first()
print(record)
print(record.keys())
print(record.count_1)

# Example 2-16. Renaming our count column
s = select([func.count(cookies.c.cookie_name).label('inventory_count')])
rp = connection.execute(s)
record = rp.first()
print(record.keys())
print(record.inventory_count)


# Filter
s = select([cookies]).where(cookies.c.cookie_name == 'chocolate chip')
print(str(s))
rp = connection.execute(s)
record = rp.first()
print(record.items())

# like
s = select([cookies]).where(cookies.c.cookie_name.like('%chocolate%'))
rp = connection.execute(s)
print(str(s))
for record in rp.fetchall():
    print(record.cookie_name)


# Operators
s = select([cookies.c.cookie_name, 'SKU-' + cookies.c.cookie_sku])
for row in connection.execute(s):
    print(row)
# ('chocolate chip', 'SKU-CC01')
# ('dark chocolate chip', 'SKU-CC02')
# ('penaut butter', 'SKU-PB01')
# ('oatmeal raisin', 'SKU-100')



INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (:cookie_name, :cookie_recipe_url, :cookie_sku, :quantity, :unit_cost)
{'cookie_name': 'chocolate chip', 'cookie_recipe_url': 'http://some.aweso.me/cookie/recipe.html', 'cookie_sku': 'CC01', 'quantity': '12', 'unit_cost': '0.05'}
(1,)
INSERT INTO cookies (cookie_name, cookie_recipe_url, cookie_sku, quantity, unit_cost) VALUES (:cookie_name, :cookie_recipe_url, :cookie_sku, :quantity, :unit_cost)
(2,)
Query_data
SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_recipe_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies
[(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 12, Decimal('0.05')), (2, 'dark chocolate chip', 'http://some.aweso.me/cookie/recipe_dark.html', 'CC02', 1, Decimal('0.75')), (3, 'penaut butter', 'http://some.aweso.me/cookie/peanut.html', 'PB01', 24, Decimal('0.25')), (4, 'oatmeal raisin', 'http://some.aweso.me/cookie/pea

C:\Users\WX847\AppData\Local\Temp\ipykernel_25540\2424340655.py:73: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  rp = connection.execute(s)
C:\Users\WX847\AppData\Local\Temp\ipykernel_25540\2424340655.py:139: SADeprecationWarning: The LegacyRow.items() method is deprecated and will be removed in a future release.  Use the Row._mapping attribute, i.e., 'row._mapping.items()'. (deprecated since: 1.4)
  print(record.items())


In [87]:
from sqlalchemy import select

s = select([cookies])

print(str(s))

rp = connection.execute(s)
results = rp.fetchall()
print(type(rp))

SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_recipe_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies
<class 'sqlalchemy.engine.cursor.LegacyCursorResult'>


In [52]:
from sqlalchemy import cast, Numeric
s = select([cookies.c.cookie_name,
            cast((cookies.c.quantity * cookies.c.unit_cost), Numeric(12, 2)).label('inv_cost')])
for row in connection.execute(s):
    print("{} -  {}".format(row.cookie_name, row.inv_cost))


chocolate chip -  0.60
dark chocolate chip -  0.75
penaut butter -  6.00
oatmeal raisin -  100.00


C:\Users\WX847\AppData\Local\Temp\ipykernel_25540\3826020240.py:4: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  for row in connection.execute(s):


In [53]:
# Example 2-21
from sqlalchemy import and_, or_, not_
s = select([cookies]).where(
    and_(
        cookies.c.quantity > 23,
        cookies.c.unit_cost < 0.40
    )
)

for row in connection.execute(s):
    print(row.cookie_name)

penaut butter


In [54]:
from sqlalchemy import and_, or_, not_
s = select([cookies]).where(
    or_(
        cookies.c.quantity.between(10, 50),
        cookies.c.cookie_name.contains('chip')
    )
)

for row in connection.execute(s):
    print(row.cookie_name)
# chocolate chip
# dark chocolate chip
# penaut butter

chocolate chip
dark chocolate chip
penaut butter


In [55]:
s = select([cookies])
rp = connection.execute(s)
for row in rp.fetchall():
    print(row)

(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 12, Decimal('0.05'))
(2, 'dark chocolate chip', 'http://some.aweso.me/cookie/recipe_dark.html', 'CC02', 1, Decimal('0.75'))
(3, 'penaut butter', 'http://some.aweso.me/cookie/peanut.html', 'PB01', 24, Decimal('0.25'))
(4, 'oatmeal raisin', 'http://some.aweso.me/cookie/peanut.html', '100', 100, Decimal('1.00'))


In [56]:
# Updating data
from sqlalchemy import update
u = update(cookies).where(cookies.c.cookie_name == "chocolate chip")
u = u.values(quantity=(cookies.c.quantity + 120))
print(str(u))
result = connection.execute(u)
print(result.rowcount)
s = select([cookies]).where(cookies.c.cookie_name=="chocolate chip")
result = connection.execute(s).first()
for key in result.keys():
    print('{:>20}: {}'.format(key, result[key]))

UPDATE cookies SET quantity=(cookies.quantity + :quantity_1) WHERE cookies.cookie_name = :cookie_name_1
1
           cookie_id: 1
         cookie_name: chocolate chip
   cookie_recipe_url: http://some.aweso.me/cookie/recipe.html
          cookie_sku: CC01
            quantity: 132
           unit_cost: 0.05


In [57]:
# Deleting Data
from sqlalchemy import delete
u = delete(cookies).where(cookies.c.cookie_name == "dark chocolate chip")
result = connection.execute(u)
print(result.rowcount)

s = select([cookies]).where(cookies.c.cookie_name == "dark chocolate chip")
result = connection.execute(s).fetchall()
print(len(result))

1
0


In [58]:
customer_list = [
    {
        'username': 'cookiemon',
        'email_address': 'mon@cookie.com',
        'phone': '111-111-1111',
        'password': 'password',
    },
    {
        'username': 'cakeeater',
        'email_address': 'cakeeater@cake.com',
        'phone': '222-222-2222',
        'password': 'password'
    },
    {
        'username': 'pieguy',
        'email_address': 'guy@pie.com',
        'phone': '333-333-3333',
        'password': 'password'
    }
]

ins = users.insert()
result = connection.execute(ins, customer_list)

In [59]:
ins = insert(orders).values(user_id=1, order_id=1)
result = connection.execute(ins)

ins = insert(line_items)
order_items = [
    {
        'order_id': 1,
        'cookie_id': 1,
        'quantity': 2,
        'extended_cost': 1.00
    },
    {
        'order_id': 1,
        'cookie_id': 3,
        'quantity': 12,
        'extended_cost': 3.00
    }
]
result = connection.execute(ins, order_items)
ins = insert(orders).values(user_id=2, order_id=2)
result = connection.execute(ins)
ins = insert(line_items)
order_items = [
    {
        'order_id':2,
        'cookie_id': 1,
        'quantity': 24,
        'extended_cost': 12.00
    },
    {
        'order_id': 2,
        'cookie_id': 4,
        'quantity': 6,
        'extended_cost': 6.00
    }
]
result = connection.execute(ins, order_items)


In [60]:
# Joins
# Example 2-25 Using join to select from multiple tables
columns = [orders.c.order_id, users.c.username, users.c.phone,
           cookies.c.cookie_name, line_items.c.quantity,
           line_items.c.extended_cost]
cookiemon_orders = select(columns)
cookiemon_orders = cookiemon_orders.select_from(orders.join(users).join(line_items).join(cookies)).where(users.c.username == 'cookiemon')

result = connection.execute(cookiemon_orders).fetchall()
for row in result:
    print(row)

# (1, 'cookiemon', '111-111-1111', 'chocolate chip', 2, Decimal('1.00'))
# (1, 'cookiemon', '111-111-1111', 'penaut butter', 12, Decimal('3.00'))

(1, 'cookiemon', '111-111-1111', 'chocolate chip', 2, Decimal('1.00'))
(1, 'cookiemon', '111-111-1111', 'penaut butter', 12, Decimal('3.00'))


C:\Users\WX847\AppData\Local\Temp\ipykernel_25540\3713429094.py:9: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  result = connection.execute(cookiemon_orders).fetchall()


In [61]:
print(str(cookiemon_orders))

SELECT orders.order_id, users.username, users.phone, cookies.cookie_name, line_items.quantity, line_items.extended_cost 
FROM orders JOIN users ON users.user_id = orders.user_id JOIN line_items ON orders.order_id = line_items.order_id JOIN cookies ON cookies.cookie_id = line_items.cookie_id 
WHERE users.username = :username_1


In [62]:
# Example 2-26 Using outerjoin to select from multiple tables
columns = [users.c.username, func.count(orders.c.order_id)]
all_orders = select(columns)
all_orders = all_orders.select_from(users.outerjoin(orders))
all_orders = all_orders.group_by(users.c.username)
result = connection.execute(all_orders).fetchall()
for row in result:
    print(row)
# ('cakeeater', 1)
# ('cookiemon', 1)
# ('pieguy', 0)

('cakeeater', 1)
('cookiemon', 1)
('pieguy', 0)


In [63]:
print(str(all_orders))

SELECT users.username, count(orders.order_id) AS count_1 
FROM users LEFT OUTER JOIN orders ON users.user_id = orders.user_id GROUP BY users.username


In [64]:
# from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, select, ForeignKey, alias

# # 创建数据库引擎和元数据对象
# engine = create_engine('sqlite:///:memory:')
# metadata = MetaData()

# # 创建 employee 表
# employee_table = Table(
#     'employee', metadata,
#     Column('id', Integer, primary_key=True),
#     Column('manager_id', None, ForeignKey('employee.id')),
#     Column('name', String(255))
# )
# # 创建表
# metadata.create_all(engine)

# # 插入一些示例数据
# with engine.connect() as conn:
#     conn.execute(employee_table.insert(), [
#         {'name': 'Fred', 'manager_id': None},
#         {'name': 'Bob', 'manager_id': 1},
#         {'name': 'Charlie', 'manager_id': 2},
#         {'name': 'David', 'manager_id': 1},
#     ])

# # 使用别名查询由名为 'Fred' 的员工管理的所有员工
# with engine.connect() as conn:
#     # 创建别名
#     employee_alias = alias(employee_table)

#     # 创建查询
#     stmt = (
#         select([employee_table.c.name])
#         .select_from(employee_table.join(employee_alias, employee_table.c.manager_id == employee_alias.c.id))
#         .where(employee_alias.c.name == 'Fred')
#     )

#     # 执行查询
#     result = conn.execute(stmt)

#     # 打印结果
#     for row in result:
#         print(f"Employee managed by Fred: {row.name}")
# # Employee managed by Fred: Bob
# # Employee managed by Fred: David


In [70]:
# Grouping
columns = [users.c.username, func.count(orders.c.order_id)]
all_orders = select(columns)
all_orders = all_orders.select_from(users.outerjoin(orders))
all_orders = all_orders.group_by(users.c.username)
result = connection.execute(all_orders).fetchall()
for row in result:
    print(row)
print(all_orders)

('cakeeater', 1)
('cookiemon', 1)
('pieguy', 0)
SELECT users.username, count(orders.order_id) AS count_1 
FROM users LEFT OUTER JOIN orders ON users.user_id = orders.user_id GROUP BY users.username


In [73]:
# Chaining
# Example 2-28
def get_orders_by_customer(cust_name):
    columns = [orders.c.order_id, users.c.username, users.c.phone,
               cookies.c.cookie_name, line_items.c.quantity,
               line_items.c.extended_cost]
    cust_orders = select(columns)
    cust_orders = cust_orders.select_from(
        users.join(orders).join(line_items).join(cookies)
    )
    cust_orders = cust_orders.where(users.c.username == cust_name)
    result = connection.execute(cust_orders).fetchall()
    return result

get_orders_by_customer('cakeeater')
# [(2, 'cakeeater', '222-222-2222', 'chocolate chip', 24, Decimal('12.00')),
#  (2, 'cakeeater', '222-222-2222', 'oatmeal raisin', 6, Decimal('6.00'))]

[(2, 'cakeeater', '222-222-2222', 'chocolate chip', 24, Decimal('12.00')),
 (2, 'cakeeater', '222-222-2222', 'oatmeal raisin', 6, Decimal('6.00'))]

In [77]:
# example 2-29 Conditional chaining
def get_orders_by_customer(cust_name, shipped=None, details=False):
    columns = [orders.c.order_id, users.c.username, users.c.phone]
    joins = users.join(orders)
    if details:
        columns.extend([cookies.c.cookie_name, line_items.c.quantity, line_items.c.extended_cost])
        joins = joins.join(line_items).join(cookies)
    cust_orders = select(columns)
    cust_orders = cust_orders.select_from(joins).where(users.c.username == cust_name)
    if shipped is not None:
        cust_orders = cust_orders.where(orders.c.shipped == shipped)
    result = connection.execute(cust_orders).fetchall()
    return result
print(get_orders_by_customer('cakeeater'))
print(get_orders_by_customer('cakeeater', details=True))
# print(get_orders_by_customer('cakeeater', shipped=True))
# get_orders_by_customer('cakeeater', shipped=False, details=True)


[(2, 'cakeeater', '222-222-2222')]
[(2, 'cakeeater', '222-222-2222', 'chocolate chip', 24, Decimal('12.00')), (2, 'cakeeater', '222-222-2222', 'oatmeal raisin', 6, Decimal('6.00'))]


In [78]:
# Raw query
rp = connection.execute("select * from orders")
print(type(rp))
result = rp.fetchall()
print(result)


<class 'sqlalchemy.engine.cursor.LegacyCursorResult'>
[(1, 1), (2, 2)]


In [88]:
from sqlalchemy import text
stmt = select([users]).where(text("username='cookiemon'"))
print(type(stmt))
rp = connection.execute(stmt)
print(type(rp))
print(rp.fetchall())
for row in rp.fetchall():
    print(row)


<class 'sqlalchemy.sql.selectable.Select'>
<class 'sqlalchemy.engine.cursor.LegacyCursorResult'>
[(1, 'cookiemon', 'mon@cookie.com', '111-111-1111', 'password', datetime.datetime(2023, 12, 28, 16, 59, 6, 995015), datetime.datetime(2023, 12, 28, 16, 59, 6, 995015))]
